In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from datetime import datetime
from pathlib import Path

import hydra
import torch
import wandb
from hydra.utils import instantiate
from omegaconf import DictConfig, OmegaConf

from opr.datasets.dataloader_factory import make_dataloaders
from opr.testing import test
from opr.training import epoch_loop

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
from hydra import initialize, compose
from hydra.utils import instantiate

with initialize(version_base=None, config_path="configs"):
    cfg = compose(config_name='config_itlp_text.yaml')  # only_text
    # cfg = compose(config_name='config_default.yaml')  # lidar + cam

In [12]:
if not cfg.general.debug and not cfg.wandb.disabled:
    config_dict = OmegaConf.to_container(cfg, resolve=True, throw_on_missing=True)
    wandb.init(
        name=cfg.wandb.run_name,
        project=cfg.wandb.project,
        settings=wandb.Settings(start_method="thread"),
        config=config_dict,
    )
    wandb.save(f"configs/{wandb.run.name}.yaml")
    run_name = wandb.run.name
else:
    run_name = "debug"

In [13]:
checkpoints_dir = (
    Path(cfg.general.checkpoints_dir) / f"{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}_{run_name}"
)
if not checkpoints_dir.exists():
    checkpoints_dir.mkdir(parents=True)

In [14]:
import os
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from joblib import dump

train_csv_path = "/home/docker_opr/Datasets/MIPT_campus/splitted/train/train.csv"

df = pd.read_csv(train_csv_path)

def train_tfidf_pca(corpus, n_components=100, base_savepath="./opr/datasets/"):
    vectorizer = TfidfVectorizer()
    vectorizer.fit(corpus)
    vectorized_corpus = vectorizer.transform(corpus).toarray()

    pca =PCA(n_components=n_components)
    pca.fit(vectorized_corpus)
    
    vectorizer_savepath = os.path.join(base_savepath, 'vectorizer.joblib')
    pca_savepath = os.path.join(base_savepath, 'pca.joblib')
    dump(vectorizer, vectorizer_savepath) 
    dump(pca, pca_savepath) 

def text_transform(self, text):
    vect_data = self.vectorizer.transform([text]).toarray()
    pca_data = self.pca.transform(vect_data)
    pca_data = torch.tensor(pca_data, dtype=torch.float32)
    return pca_data

In [15]:
if "text" in cfg.general.modalities:
    corpus = np.hstack((df["back_description"], df["front_description"]))
    train_tfidf_pca(corpus)

In [16]:
print("=> Instantiating model...")
model = instantiate(cfg.model)

print("=> Instantiating loss...")
loss_fn = instantiate(cfg.loss)

print("=> Making dataloaders...")
dataloaders = make_dataloaders(
    dataset_cfg=cfg.dataset.dataset,
    batch_sampler_cfg=cfg.dataset.sampler,
    num_workers=cfg.dataset.num_workers,
)

=> Instantiating model...
=> Instantiating loss...
=> Making dataloaders...


In [17]:
cfg

{'general': {'debug': False, 'seed': 31299, 'checkpoints_dir': 'checkpoints/', 'device': 'cuda', 'num_workers': 4, 'batch_expansion_th': 0.7, 'modalities': ['text'], 'epochs': 40}, 'loss': {'_target_': 'opr.losses.MultimodalTripletMarginLoss', 'margin': 0.2, 'swap': True, 'distance': {'_target_': 'pytorch_metric_learning.distances.LpDistance', 'normalize_embeddings': False, 'collect_stats': True}, 'miner': {'_target_': 'opr.miners.HardTripletMiner', 'distance': {'_target_': 'pytorch_metric_learning.distances.LpDistance', 'normalize_embeddings': False}, 'modalities': '${loss.modalities}'}, 'reducer': {'_target_': 'pytorch_metric_learning.reducers.AvgNonZeroReducer', 'collect_stats': True}, 'modalities': ['text'], 'weights': [1]}, 'wandb': {'disabled': False, 'project': 'OpenPlaceRecognition', 'run_name': 'only_text'}, 'model': {'_target_': 'opr.models.base_models.ComposedModel', 'image_module': None, 'cloud_module': None, 'fusion_module': None, 'text_module': {'_target_': 'opr.models.ba

In [18]:
print("=> Instantiating optimizer...")
params_list = []
for modality in cfg.general.modalities:
    params_list.append(
        {
            "params": getattr(model, f"{modality}_module").parameters(),
            "lr": cfg.optimizer.learning_rates[f"{modality}_lr"],
        }
    )
optimizer = instantiate(cfg.optimizer.fn, params=params_list)
print("Instantiating scheduler...")
scheduler = instantiate(cfg.scheduler, optimizer=optimizer)

model = model.to(cfg.general.device)

=> Instantiating optimizer...
Instantiating scheduler...


In [15]:
epoch

79

In [19]:
# ==========> TRAIN LOOP:

best_recall_at_1 = 0.0

for epoch in range(cfg.general.epochs):
# for epoch in range(120, 180):
    print(f"\n\n=====> Epoch {epoch+1}:")
    # TODO: resolve mypy typing here
    train_batch_size = dataloaders["train"].batch_sampler.batch_size  # type: ignore

    print("\n=> Training:\n")

    train_stats, train_rate_non_zero = epoch_loop(
        dataloader=dataloaders["train"],
        model=model,
        loss_fn=loss_fn,
        optimizer=optimizer,
        scheduler=scheduler,
        phase="train",
        device=cfg.general.device,
    )

    print(f"\ntrain_rate_non_zero = {train_rate_non_zero}")

    batch_expansion_th = cfg.general.batch_expansion_th
    if batch_expansion_th is not None:
        if batch_expansion_th == 1.0:
            print("Batch expansion rate is set to every epoch. Increasing batch size.")
            # TODO: resolve mypy typing here
            dataloaders["train"].batch_sampler.expand_batch()  # type: ignore
        elif train_rate_non_zero is None:
            print(
                "\nWARNING: 'batch_expansion_th' was set, but 'train_rate_non_zero' is None. ",
                "The batch size was not expanded.",
            )
        elif train_rate_non_zero < batch_expansion_th:
            print(
                "Average non-zero triplet ratio is less than threshold: ",
                f"{train_rate_non_zero} < {batch_expansion_th}",
            )
            # TODO: resolve mypy typing here
            dataloaders["train"].batch_sampler.expand_batch()  # type: ignore

    print("\n=> Validating:\n")

    val_stats, val_rate_non_zero = epoch_loop(
        dataloader=dataloaders["val"],
        model=model,
        loss_fn=loss_fn,
        optimizer=optimizer,
        phase="val",
        device=cfg.general.device,
    )

    print(f"\nval_rate_non_zero = {val_rate_non_zero}")

    print("\n=> Testing:\n")

    recall_at_n, recall_at_one_percent, mean_top1_distance = test(
        model=model,
        descriptor_key="text",
        dataloader=dataloaders["test"],
        device=cfg.general.device,
    )

    stats_dict = {}
    stats_dict["test"] = {
        "mean_top1_distance": mean_top1_distance,
        "recall_at_1%": recall_at_one_percent,
        "recall_at_1": recall_at_n[0],
        "recall_at_3": recall_at_n[2],
        "recall_at_5": recall_at_n[4],
        "recall_at_10": recall_at_n[9],
    }
    stats_dict["train"] = train_stats
    stats_dict["train"]["batch_size"] = train_batch_size
    stats_dict["val"] = val_stats

    # saving checkpoints
    checkpoint_dict = {
        "epoch": epoch + 1,
        "stats_dict": stats_dict,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
    }
    torch.save(checkpoint_dict, checkpoints_dir / f"epoch_{epoch+1}.pth")
    if recall_at_n[0] > best_recall_at_1:
        print("Recall@1 improved!")
        torch.save(checkpoint_dict, checkpoints_dir / "best.pth")
        best_recall_at_1 = recall_at_n[0]
        
    wandb.log({'Test recall_at_1': recall_at_n[0],
                'Test Mean Recall@1%': recall_at_one_percent,
                    'Test Mean top-1 distance': mean_top1_distance,
                    'batch_size': train_stats['batch_size'],
                    'train_total_loss': train_stats['total_loss'],
                    'val_total_loss': val_stats['total_loss']})



=====> Epoch 1:

=> Training:



train: 100%|██████████| 90/90 [00:07<00:00, 12.00it/s]


train_rate_non_zero = 1.0

=> Validating:




val: 100%|██████████| 33/33 [00:01<00:00, 18.99it/s]



val_rate_non_zero = 1.0

=> Testing:



...


train_rate_non_zero = 0.9368370980870979

=> Validating:




val: 100%|██████████| 33/33 [00:01<00:00, 16.89it/s]



val_rate_non_zero = 1.0

=> Testing:



Calculating metrics: 100%|██████████| 6/6 [00:00<00:00, 16.13it/s]

Mean Recall@N:
[0.37854936 0.51501274 0.58377464 0.6325714  0.66697025 0.69788195
 0.72846038 0.7503656  0.77364978 0.79229798 0.80526232 0.81715741
 0.82770661 0.8369141  0.84692055 0.85744039 0.8650269  0.87180342
 0.87774485 0.88504855 0.89046093 0.89559856 0.900739   0.90507166
 0.90994851]
Mean Recall@1% = 0.6978819539999169
Mean top-1 distance = 0.15801929631771264
Recall@1 improved!
